### Importazioni

In [10]:
from dotenv import dotenv_values
import json
import requests

## Estraggo le coordinate dai POI

In [47]:
# Apro il file e leggo il contenuto
with open('DICTIONARYTOTALE.txt', 'r') as file:
    contenuto = file.read()

In [48]:
# Decodifico il contenuto JSON in un dizionario Python
dizionario = json.loads(contenuto)

# Lista per le coordinate
coordinate = []

# Estraggo le longitudini e latitudini e le aggiungo alla lista coordinate
for elemento in dizionario.values():
    latitudine = elemento.get('lat')
    longitudine = elemento.get('lon')
    if latitudine is not None and longitudine is not None:
        coordinate.append((latitudine, longitudine))

# Stampo la lista delle coordinate
print(coordinate)

[(42.68827, 12.549764), (42.683333, 12.55), (42.690376, 12.54649), (42.691138, 12.54658), (42.69067, 12.546327), (42.70388888888889, 12.574444444444444), (42.698055555555555, 12.531388888888888), (42.673620035730664, 12.5873151), (42.673169, 12.603323), (42.656894, 12.536004), (42.7100669, 12.4951428), (42.70867, 12.48504), (42.664167, 12.508611), (42.71342, 12.50648), (42.711472, 12.611841), (42.708, 12.621), (42.65, 12.55), (42.64827, 12.637932), (42.73033, 12.58681), (42.7847, 12.0356), (42.786111, 12.037778), (42.772684, 12.039017), (42.771667, 12.018611), (42.804302, 12.071065), (42.75527777777778, 12.001666666666667), (42.80222222222222, 11.983888888888888), (42.833333, 12.066667), (42.835532, 12.065821), (42.816667, 11.966667), (42.755278, 11.945833), (42.7919, 11.9175), (42.7805, 11.9067), (42.587742, 12.295626), (42.583333, 12.3), (42.591907, 12.25983), (42.574862, 12.32824), (42.616667, 12.3), (42.566667, 12.333333), (42.62388, 12.29671), (42.621667, 12.254444), (42.546496, 1

In [49]:
print(str(len(coordinate)) + " POI")

3141 POI


In [50]:
poche_coordinate = coordinate[:5]

print(poche_coordinate)

[(42.68827, 12.549764), (42.683333, 12.55), (42.690376, 12.54649), (42.691138, 12.54658), (42.69067, 12.546327)]


## Recupero la chiave API

In [51]:
dotenv = dotenv_values()
api_key = dotenv.get("API_KEY");

print(api_key)

8A20587981B148ADBBF3BED91ACFC937


## Estraggo i POI presenti su TripAdvisor

In [55]:
# Creo un insieme per memorizzare gli ID già presenti nel file
id_presenti = set()

# Controllo se il file esiste
import os
if os.path.exists('poi.txt'):
    # Se il file esiste, leggo il contenuto attuale del file e memorizzo gli ID
    with open('poi.txt', 'r') as file:
        for line in file:
            id, _ = line.strip().split(': ')
            id_presenti.add(id)

# Creo la lista vuota di POI da salvare
poi = []

# Mi salvo nella lista l'id e il nome del POI solo se l'id non è già presente
for coordinate in poche_coordinate:
    url = "https://api.content.tripadvisor.com/api/v1/location/nearby_search?latLong=" + str(coordinate[0]) + "%2C" + str(coordinate[1]) + "&key=" + api_key + "&category=attractions&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
   
    POI = response.json()

    # Estraggo l'id e il nome di ciascun POI e li aggiungo alla lista di POI se l'id non è già presente
    for elemento in POI['data']:
        nome = elemento.get('name')
        id = elemento.get('location_id')
        citta = elemento.get('address_obj')
        if nome is not None and id is not None and id not in id_presenti:
            poi.append((id, nome, citta))
            # Aggiungo l'id all'insieme degli ID presenti
            id_presenti.add(id)

# Apro il file in modalità scrittura in modalità append
with open('poi.txt', 'a') as file:
    # Itero sulla lista di POI e scrivo ciascun POI nel file solo se l'id non è già presente
    for id, nome, citta in poi:
        file.write(f"{id}: {nome}, {citta}\n")

print(poi)
    

[('8666261', 'Palazzo Cesi', {'street1': 'Piazza Federico Cesi 3', 'street2': '', 'city': 'Acquasparta', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05021', 'address_string': 'Piazza Federico Cesi 3, 05021, Acquasparta Italia'}), ('13436625', 'Chiesa di Santa Cecilia', {'street1': 'Via Colonna 79', 'street2': '', 'city': 'Acquasparta', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05021', 'address_string': 'Via Colonna 79, 05021, Acquasparta Italia'})]


In [3]:
# Inizializzo un dizionario vuoto per memorizzare gli ID e i nomi dei POI
poi_dict = {}

# Leggo il contenuto del file riga per riga
with open('poi.txt', 'r') as file:
    for line in file:
        # Divido la riga usando ':' come separatore
        id, nome = line.strip().split(': ')
        # Aggiungo l'id e il nome al dizionario
        poi_dict[id] = nome

# Ora poi_dict conterrà gli ID come chiavi e i nomi dei POI come valori
print(poi_dict)

{'8666261': 'Palazzo Cesi', '13436625': 'Chiesa di Santa Cecilia'}


In [4]:
for elemento in poi_dict:
    print(elemento)

8666261
13436625


In [20]:
poi_dict

{'8666261': 'Palazzo Cesi', '13436625': 'Chiesa di Santa Cecilia'}

### Salvo i POI in un dizionario JSON

In [22]:
# salvo il dizionario in un file json
dizionario_json = "poi.json"

# Salvataggio del dizionario nel file JSON
with open(dizionario_json, "w") as file:
    json.dump(poi_dict, file)

## Recupero le recensioni relative ai POI estratti

In [36]:
# Dizionario per memorizzare le recensioni per ogni location
recensioni_per_location = {}

# Itero sui POI e recupera le recensioni
for elemento in poi_dict:
    url = "https://api.content.tripadvisor.com/api/v1/location/" + elemento + "/reviews?key=" + api_key + "&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    REC = response.json()

    # Estraggo le recensioni di ciascun POI e le aggiungo al dizionario
    for elemento in REC['data']:
        id_rec = elemento.get('id')
        id_loc = elemento.get('location_id')
        rating = elemento.get('rating')
        recensione = elemento.get('text')

        # Se la location_id non è già presente nel dizionario, l'aggiungo come chiave con una lista vuota come valore
        if id_loc not in recensioni_per_location:
            recensioni_per_location[id_loc] = {}

        # Aggiungo la recensione al dizionario nidificato utilizzando l'id della recensione come chiave
        recensioni_per_location[id_loc][id_rec] = {'rating': rating, 'recensione': recensione}

# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_per_location.json', 'w') as file:
    json.dump(recensioni_per_location, file)

In [130]:
# Apro il file in modalità lettura
with open('recensioni_per_location.json', 'r') as file:
    recensioni = file.read();
    # Decodifico il contenuto JSON in un dizionario Python

print(recensioni)
print(len(recensioni))
dizionario_recensioni = json.loads(recensioni)

{"8666261":{"917883573":{"rating":4, "recensione":"Palazzo Cesi di Acquasparta \u00e9 il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che d\u00e0 sulla piazza principale, si presenta, nell\u2019ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL\u2019interno \u00e9 impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo \u00e9 edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido \u00e9 anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, si

In [131]:
dizionario_recensioni

{'8666261': {'917883573': {'rating': 4,
   'recensione': 'Palazzo Cesi di Acquasparta é il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che dà sulla piazza principale, si presenta, nell’ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL’interno é impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo é edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido é anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, sia al piano terra che al prim

In [132]:
type(dizionario_recensioni)

dict

In [133]:
# recupero anche il dizionario con solo chiave univoca della location e il nome
with open('poi.json', 'r') as file:
    poi_json = file.read()
    dizionario_poi = json.loads(poi_json)

In [134]:
type(dizionario_poi)

dict

In [135]:
dizionario_poi.items()

dict_items([('8666261', 'Palazzo Cesi'), ('13436625', 'Chiesa di Santa Cecilia')])

In [136]:
dizionario_recensioni

{'8666261': {'917883573': {'rating': 4,
   'recensione': 'Palazzo Cesi di Acquasparta é il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che dà sulla piazza principale, si presenta, nell’ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL’interno é impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo é edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido é anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, sia al piano terra che al prim

In [ ]:
for items in dizionario_poi.items():
    for idloc, recensione_info in dizionario_recensioni.items():
        recensione_info = {'POI': "nome_poi", **recensione_info}

In [139]:
for items in dizionario_poi.items():
    for recensione in dizionario_recensioni.items():
        print(items[1])

Palazzo Cesi
Palazzo Cesi
Chiesa di Santa Cecilia
Chiesa di Santa Cecilia


In [148]:
for id_rec, recensione_info in dizionario_recensioni.items():
    # Controllo se l'id della recensione corrisponde a un id del POI
    if id_rec in dizionario_poi:
        # Ottengo il nome del POI associato all'id del POI
        nome_poi = dizionario_poi[id_rec]
        # Aggiungo il nome del POI all'inizio della recensione
        recensione_info = {'POI': nome_poi, **recensione_info}
        # Aggiorno il dizionario delle recensioni con la recensione aggiornata
        dizionario_recensioni[id_rec] = recensione_info

# Ora dizionario_recensioni contiene il campo 'POI' aggiunto all'inizio di ciascuna recensione



In [149]:
dizionario_recensioni


{'8666261': {'POI': 'Palazzo Cesi',
  '917883573': {'rating': 4,
   'recensione': 'Palazzo Cesi di Acquasparta é il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che dà sulla piazza principale, si presenta, nell’ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL’interno é impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo é edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido é anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, sia a

In [151]:
# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_complete.json', 'w') as file:
    json.dump(dizionario_recensioni, file)